## Function Calling

The main purpose of Function Calling in large models is to extend their capabilities, allowing them to integrate a wider range of information and functions when generating text, making decisions, or performing tasks. By defining and calling external functions, large models can handle more complex tasks, such as performing mathematical operations, placing orders, or querying databases, thereby improving their practicality and flexibility.


### 🌟Using LlamaIndex

LlamaIndex has developed an easy-to-use method for Function Calling. Let's use `yi-large` for an example.

First, let's install the dependencies. Follow along step by step!


In [ ]:
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-llms-yi

Make sure the installation is complete, and then we'll get started.

Load the dependencies.

In [ ]:
from typing import Any

from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.llms.yi import Yi

from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

Configure the model. Here, we load the open-source model from Huggingface. You can also download `Yi-1.5-34B-Chat` directly from Huggingface.


In [ ]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name='/model/Yi-1.5-9B-Chat', # Load local model
    model_name='/model/Yi-1.5-9B-Chat', # Load local model
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16},
)

Define the tools. Here, we define three functions: `add`, `subtract`, and `multiply`.


In [ ]:
# Define the tools
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the result."""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and return the result."""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two integers and return the result."""
    return a - b

multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

Create the agent


In [ ]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, subtract_tool], llm=llm, verbose=True)

Make sure everything runs correctly, and then we'll start a conversation


In [ ]:
while True:
    user_input = input("user>>") # "(1+2)*10"
    agent.chat(user_input)

Let's see the results


``````
user>>(1+2)*10
Thought: The user's question is in English. I need to use a combination of the 'add' and 'multiply' tools to solve the mathematical expression (1+2)*10.
Action: add
Action Input: {'a': 1, 'b': 2}
Observation: 3
Thought: I have the result of the 'add' operation, which is 3. Now I need to multiply this result by 10 to complete the expression (1+2)*10.
Action: multiply
Action Input: {'a': 3, 'b': 10}
Observation: 30
Thought: I have the result of the 'multiply' operation, which is 30. This is the final result of the expression (1+2)*10.
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of the expression (1+2)*10 is 30.
``````